In [1]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense, Embedding, Lambda, TimeDistributed, \
                                    Add, Conv1D, Dropout, Concatenate, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard

from model import create_model 

Using TensorFlow backend.


In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU') 
for physical_device in physical_devices: 
    tf.config.experimental.set_memory_growth(physical_device, True)

print(physical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
import numpy as np
import json
from tqdm.auto import tqdm
import time
import sys

In [4]:
!pwd

/home/sweet/1-workdir/nlp_attention/en_vi_attention_nlp/src


In [5]:
num_layers=2
num_multi_heads=3
d_k=64
d_v=64
d_model=256
optimizer="adam"
null_token_value=0
source_vocab_size = 48114
target_vocab_size = 22468
share_word_embedding=False
MAXIMUM_TEXT_LENGTH = 866

In [6]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [7]:
training_records="/home/sweet/1-workdir/nlp_attention/en_vi_attention_nlp/src/train.tfrecords"
validating_records="/home/sweet/1-workdir/nlp_attention/en_vi_attention_nlp/src/test.tfrecords"

raw_training_set = tf.data.TFRecordDataset(training_records)
raw_validating_set = tf.data.TFRecordDataset(validating_records)


feature_description = {
    'input': tf.io.FixedLenFeature([], tf.string),
    'target': tf.io.FixedLenFeature([], tf.string)
}

def _parse_record_function(example_proto):
    # Parse the input tf.Example proto using the dictionary above.
    features = tf.io.parse_single_example(example_proto, feature_description)
    X = tf.io.decode_raw(features['input'], np.int32)
    Y = tf.io.decode_raw(features['target'], np.int32)
    return X[3:], Y[3:]

In [8]:
with tf.device('/cpu:0'):
    BUFFER_SIZE_TRAIN = 133317
    BATCH_SIZE_TRAIN = 32
    N_STEPS_PER_EPOCH_TRAIN = int(np.ceil(BUFFER_SIZE_TRAIN/BATCH_SIZE_TRAIN))

    train_dataset = raw_training_set.map(_parse_record_function)\
                                    .repeat()\
                                    .batch(batch_size=BATCH_SIZE_TRAIN)\
                                    .prefetch(buffer_size=AUTOTUNE)
    train_gen = train_dataset.__iter__()

In [9]:
with tf.device('/cpu:0'):
    BUFFER_SIZE_VAL = 2821
    BATCH_SIZE_VAL = 32
    N_STEPS_PER_EPOCH_VAL = int(np.ceil(BUFFER_SIZE_VAL/BATCH_SIZE_VAL))

    validation_dataset = raw_validating_set.map(_parse_record_function)\
                                        .batch(batch_size=BATCH_SIZE_VAL)\
                                        .prefetch(buffer_size=AUTOTUNE)
    val_gen = validation_dataset.__iter__()

In [10]:
example_input_batch, example_target_batch = next(train_gen)
example_input_batch.shape, example_target_batch.shape

(TensorShape([32, 865]), TensorShape([32, 865]))

In [11]:
with tf.device('/cpu:0'):
    training_model, inference_model = create_model(source_vocab_size, target_vocab_size, 
                                                   MAXIMUM_TEXT_LENGTH,
                                                   n=num_layers, d_model=256, h=num_multi_heads, 
                                                   optimizer=Adam(0.001, 0.9, 0.98, epsilon=1e-9))
#     training_model.summary()

In [12]:
EPOCHS = 15
pbar = tqdm(total=N_STEPS_PER_EPOCH_TRAIN)
for epoch in range(EPOCHS):
    i = 0
    pbar.reset()
    for inp, targ in train_gen:
        hist = training_model.fit([inp, targ])
        pbar.set_postfix(cost = hist.history['loss'][-1])
        pbar.update(1)
        i = i+1

1/1 [==============================] - 0s 778us/step - loss: 6.4698


1/1 [==============================] - 0s 744us/step - loss: 6.2483


KeyboardInterrupt: 